In [6]:
##Esse código converte kml, cvs e shp em ee.Geometry


import ee


ee.Initialize()  

import eemont, geemap


import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon



import numpy as np
from functools import reduce

### *** Leitura e apresentação do formato KML *** 

In [7]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gdf = gpd.read_file('C:\\Users\\jabre\\Documents\\merx\\merx\\Croquis\\coordenadas_4160671_1.kml', encoding="utf-8")
g = [i for i in gdf.geometry]
features=[]

for i in range(len(g)):
                g = [i for i in gdf.geometry]
                x,y = g[i].exterior.coords.xy
                cords = np.dstack((x,y)).tolist()

                g=ee.Geometry.Polygon(cords)
                feature = ee.Feature(g)
                features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)

CAR =ee_object

done


In [8]:
##LEMBRAR DE adicionar máscara de agricultura antes de selecionar o ponto
lat_long = CAR.geometry().centroid()

start_date = '2019-01-01'
end_date = '2022-09-11'

######################## NDVI - MAPAS ########################


S2_out = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-10-01', '2021-11-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)        
S2_nov = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-11-01', '2021-12-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_dez = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-12-01', '2022-01-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_jan = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-01-01', '2022-02-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_fev = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-02-01', '2022-03-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_mar = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-03-01', '2022-04-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_abr = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-04-01', '2022-05-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)

Mapa = geemap.Map()
Mapa.add_basemap("SATELITE")
Mapa.centerObject(ee.FeatureCollection(CAR),12)

Mapa.addLayer(S2_out.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-out")
Mapa.addLayer(S2_nov.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-nov")
Mapa.addLayer(S2_dez.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-dez")
Mapa.addLayer(S2_jan.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-jan")
Mapa.addLayer(S2_fev.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-fev")
Mapa.addLayer(S2_mar.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-marco")
Mapa.addLayer(S2_abr.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-abril")


Mapa



Map(center=[-24.51121086303618, -53.948645031752314], controls=(WidgetControl(options=['position', 'transparen…

### *** Leitura e apresentação dos formato SHP e CSV *** 

In [4]:
#Feature to EE
def feature2ee(file):
    
    #Exception handler
    try:
        
        #check if the file is shapefile or CSV
        if file.endswith('.shp'):
            gdf = gpd.read_file(file, encoding="utf-8")
            g = [i for i in gdf.geometry]
            features=[]
            
            
            #for Polygon geo data type
            if (gdf.geom_type[0] == 'Polygon'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()

                    g=ee.Geometry.Polygon(cords)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)
                
                return ee_object
            
                
            #for LineString geo data type   
            elif (gdf.geom_type[0] == 'LineString'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)

                    g=ee.Geometry.LineString(double_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)
                
                return ee_object
            
            
            #for Point geo data type
            elif (gdf.geom_type[0] == 'Point'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)
                    single_list = reduce(lambda x,y: x+y, double_list)
                    
                    g=ee.Geometry.Point(single_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print("done")

                ee_object = ee.FeatureCollection(features)
                
                return ee_object
            
            
        #check if the file is shapefile or CSV
        #for CSV we need to have file with X and Y
        elif file.endswith('.csv'):
            df = pd.read_csv(file)
            features=[]
            for i in range(df.shape[0]):
                x,y = df.x[i],df.y[i]
                latlong =[x,y]
                g=ee.Geometry.Point(latlong) 
                feature = ee.Feature(g)
                features.append(feature)
            print("done")
                
            ee_object = ee.FeatureCollection(features)
            return ee_object
            
        
    except:
        
        print("Wrong file format")



In [5]:
#shapefile
fp =  "C:\\Users\\jabre\\Downloads\\shape\\to.shp"

#call function
polygon = feature2ee(fp)

done


In [10]:
CAR = polygon

lat_long = CAR.geometry().centroid()

start_date = '2019-01-01'
end_date = '2022-09-11'

######################## NDVI - MAPAS ########################


S2_out = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-10-01', '2021-11-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)        
S2_nov = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-11-01', '2021-12-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_dez = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2021-12-01', '2022-01-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_jan = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-01-01', '2022-02-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_fev = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-02-01', '2022-03-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_mar = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-03-01', '2022-04-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)
S2_abr = ee.ImageCollection('COPERNICUS/S2_SR').spectralIndices(['NDVI']).filterBounds(CAR).filterDate('2022-04-01', '2022-05-01').maskClouds(prob = 70).mean().select('NDVI').clip(CAR)

Mapa = geemap.Map()
Mapa.add_basemap("SATELITE")
Mapa.centerObject(ee.FeatureCollection(CAR),12)

Mapa.addLayer(S2_out.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-out")
Mapa.addLayer(S2_nov.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-nov")
Mapa.addLayer(S2_dez.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-dez")
Mapa.addLayer(S2_jan.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-jan")
Mapa.addLayer(S2_fev.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-fev")
Mapa.addLayer(S2_mar.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-marco")
Mapa.addLayer(S2_abr.select('NDVI'), {'min':0,'max':0.8,'palette':['red', 'blue', 'green','yellow']}, "NDVI-abril")

Mapa

Map(center=[-10.116651137546993, -48.30585539699571], controls=(WidgetControl(options=['position', 'transparen…